In [ ]:
# Copyright (C) 2016-2018 by Jakob J. Kolb at Potsdam Institute for Climate
# Impact Research
#
# Contact: kolb@pik-potsdam.de
# License: GNU AGPL Version 3

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from plot_routines import plot_routines
from scipy.interpolate import interp1d
import seaborn

In [ ]:
# Loading the data into a plotting tool class.
micro_path = '/home/jakob/Project_Divestment/PyDivestment/output_data/P3o2/micro/mean.h5'
macro_path = '/home/jakob/Project_Divestment/PyDivestment/output_data/P3o2/aggregate/mean.h5'
with pd.HDFStore(micro_path) as store:
    micro_data = store.select('dat', 'approximate=1 & test=False & sample=0 & xi=0.1 & tau>0.5')
with pd.HDFStore(macro_path) as store:
    macro_data = store.select('dat', 'approximate=2 & test=False & sample=0 & xi=0.1 & tau>0.5')
    
for df in [micro_data, macro_data]:
    df.index = df.index.droplevel(['approximate', 'test', 'sample', 'xi', 'kappa_c'])
    print(df.index.levels)
    df.index.names = [r'$\tau$',r'$\phi$', 'tsteps']

In [ ]:
macro_data.head()

In [ ]:
def l_2_distance(d1, d2, observable=None):
    
    d1_data = d1[[observable]]
    d2_data = d2[[observable]]
    
    names = d1.index.names
    print(names)
    indices = {ax: list(d1.index.levels[names.index(ax)].values) for ax in names[:2]}
    print(indices)
    output = np.zeros((len(indices[names[0]]),len(indices[names[1]])))
    
    for i, n0 in enumerate(indices[names[0]]):
        for j, n1 in enumerate(indices[names[1]]):
            levels = names[:2]
            tkey = [n0, n1]
    
            # get values and times from dataframes
            d1_slice = d1_data.xs(key=tkey, level=levels).values.T[0][:-1]
            #d1_times = d1_data.xs(key=tkey, level=levels).index.values[:-1]

            d2_slice = d2_data.xs(key=tkey, level=levels).values.T[0][:-1]
            #d2_times = d2_data.xs(key=tkey, level=levels).index.values[:-1]
            
            # interpolate to get values on equal timepoints
            #d1_ipl = interp1d(d1_times, d1_slice)
            #d2_ipl = interp1d(d2_times, d2_slice)

            #samples = 200
            #times = np.linspace(d1_times[0], min(d1_times[-1], d2_times[-1]), samples)
            #d1_ipl_data = d1_ipl(times)
            #d2_ipl_data = d2_ipl(times)
            d1_ipl_data = d1_slice
            d2_ipl_data = d2_slice
            dd = d1_ipl_data - d2_ipl_data
            l2 = np.sqrt(sum(dd*dd))
            output[i, j] = l2
            
    return output, indices, names[:2]

In [ ]:
levels=['eps']
key=[0.05]

data, indices, ax_names = l_2_distance(micro_data, macro_data, observable='z')

fig = plt.figure()
ax = fig.add_subplot(111)

seaborn.heatmap(data,
                yticklabels=indices[ax_names[0]],
                xticklabels=[round(x, 3) for x in indices[ax_names[1]]],
                ax=ax,
                cbar_kws={'label': '$L_2$ distance'})
ax.set_ylabel(ax_names[0])
ax.set_xlabel(ax_names[1])

fig = ax.get_figure()
fig.tight_layout()
fig.savefig('heatmap.png')
fig.savefig('heatmap.pdf')